In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import os
import torch.utils.data
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.nn import functional as F
import numpy as np
from torchvision. datasets import ImageFolder
from torchvision.utils import save_image
%matplotlib inline
cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')
if not os.path.exists("save_image"):
    os.mkdir("save_image")
if not os.path.exists("asset"):
    os.mkdir("asset")
    
batch_size=64
z_dim=100
train_image=ImageFolder("datasets/train/",
                        transform=transforms.Compose([
                        transforms.Scale((64,64)),
                        transforms.ToTensor()]
                            ))

train_loader=DataLoader(train_image,batch_size=batch_size,shuffle=True)



In [2]:
class Discriminator(nn.Module):
    def __init__(self,input_channels,image_size):
        super(Discriminator,self).__init__()
        self.input_channels=input_channels
        self.image_size=image_size
        #(input_channels,image_size,image_size)=(3,64,64)
        self.discriminate=nn.Sequential(
            nn.Conv2d(input_channels,64,kernel_size=4,stride=2,padding=1),
            nn.LeakyReLU(),
            #(64,32,32)
            
            nn.Conv2d(64,128,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            #(128,16,16)
            
            nn.Conv2d(128,256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            #(256,8,8)
            
            nn.Conv2d(256,512,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU()
            #(512,4,4),
            )
        
        self.dense=nn.Sequential(
            nn.Linear(4*4*512,1),
            nn.Sigmoid())
    def forward(self,x):
        batch_size=x.size()[0]
        d_pre=self.discriminate(x)
        d_pre=d_pre.view(batch_size,-1)
        out=self.dense(d_pre)
        
        return out

In [3]:
class Generator(nn.Module):
    def __init__(self,output_channels,z_dim):
        super(Generator,self).__init__()
        self.output_channels=output_channels
        self.z_dim=z_dim
        
        self.dense=nn.Sequential(
            nn.Linear(self.z_dim,4*4*1024)    
            )
        self.generate=nn.Sequential(
            nn.ConvTranspose2d(1024,512,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            #(512,8,8)
            
            nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            #(256,16,16)
            
            nn.ConvTranspose2d(256,128,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128,self.output_channels,kernel_size=4,stride=2,padding=1),
            nn.Sigmoid()
            )
        
        
        
    def forward(self,z):
        g_pre=self.dense(z)
        g_pre=g_pre.view(-1,1024,4,4)
        
        out=self.generate(g_pre)
        return out

In [8]:
G=Generator(3,z_dim)
D=Discriminator(3,batch_size)

if cuda:
    g_param=torch.load("asset/G.pth")
    d_param=torch.load("asset/D.pth")
else:
    g_param=torch.load("asset/G.pth",map_location=lambda storage, loc:storage)
    d_param=torch.load("asset/D.pth",map_location=lambda storage, loc:storage)
G.load_state_dict(g_param)
D.load_state_dict(d_param)
    


In [13]:
z=torch.rand(batch_size,z_dim)
if cuda:
    z=z.cuda()
generated_image=G(z).data.cpu()
save_image(generated_image,"save_image/inspect.png")
print("save_image!!")

save_image!!
